In [ ]:
import serial
import serial.tools.list_ports
import time

data_list = []  # store parsed values
PREFERRED_COM = "COM4"  # change this to your preferred port

def handle_data(line):
    """Parse line from ESP32 and store in data_list"""
    try:
        values = list(map(int, line.strip('[]').split(',')))
        push   = values[0:4]
        toggle = values[4:8]
        analog = values[8:14]
        global data_list
        data_list = [push, toggle, analog]
        print(analog[-2:])
    except:
        pass

# ======= Auto-find ESP32 serial port =======
def find_esp32_port():
    ports = list(serial.tools.list_ports.comports())

    # 1️⃣ Try preferred COM first
    if PREFERRED_COM:
        for p in ports:
            if p.device == PREFERRED_COM:
                try:
                    ser = serial.Serial(p.device, 115200, timeout=1)
                    ser.flushInput()
                    time.sleep(1)
                    if ser.in_waiting:
                        line = ser.readline().decode('utf-8', errors='ignore').strip()
                        if line.startswith('[') and line.endswith(']'):
                            ser.close()
                            return p.device
                    ser.close()
                except:
                    pass

    # 2️⃣ Try name-based scan (e.g., CP210x / CH340)
    for p in ports:
        desc = (p.description + str(p.manufacturer)).lower()
        if any(k in desc for k in ["cp210", "ch340", "usb serial"]):
            try:
                print('found by name')
                ser = serial.Serial(p.device, 115200, timeout=1)
                ser.flushInput()
                time.sleep(1)
                if ser.in_waiting:
                    line = ser.readline().decode('utf-8', errors='ignore').strip()
                    if line.startswith('[') and line.endswith(']'):
                        print('connected')
                        ser.close()
                        return p.device
                ser.close()
            except:
                continue

    # 3️⃣ Fallback: try all remaining ports
    for p in ports:
        try:
            ser = serial.Serial(p.device, 115200, timeout=1)
            ser.flushInput()
            time.sleep(1)
            if ser.in_waiting:
                line = ser.readline().decode('utf-8', errors='ignore').strip()
                if line.startswith('[') and line.endswith(']'):
                    ser.close()
                    return p.device
            ser.close()
        except:
            continue

    return None

# ======= Read data continuously with reconnect =======
def read_loop():
    ser = None
    esp_port = None

    while True:
        try:
            if ser is None or not ser.is_open:
                esp_port = find_esp32_port()
                if esp_port:
                    print(f"Connecting to ESP32 on {esp_port}...")
                    ser = serial.Serial(esp_port, 115200, timeout=1)
                    time.sleep(1)  # wait for ESP32 reset
                else:
                    print("ESP32 not found. Retrying...")
                    # time.sleep(1)
                    continue

            if ser.in_waiting:
                line = ser.readline().decode('utf-8', errors='ignore').strip()
                if line:
                    handle_data(line)
            time.sleep(0.03)

        except (serial.SerialException, OSError) as e:
            print(f"Connection lost: {e}. Re-scanning ports...")
            try:
                if ser:
                    ser.close()
            except:
                pass
            ser = None
            time.sleep(0.1)

        except KeyboardInterrupt:
            print("Stopped by user")
            if ser:
                ser.close()
            break

# ======= Main =======
if __name__ == "__main__":
    read_loop()


found by name
connected
Connecting to ESP32 on COM3...
[]
[2006, 1497]
[2003, 1498]
[2000, 1499]
[2145, 1495]
[2000, 1499]
[1996, 1497]
[2002, 1502]
[2007, 1497]
[2000, 1494]
[2003, 1501]
[2003, 1497]
[2006, 1499]
[2007, 1498]
[2009, 1495]
[2003, 1493]
[1999, 1495]
[2005, 1498]
[2012, 1495]
[1986, 1498]
[2001, 1497]
[2000, 1495]
[2000, 1443]
[2001, 1493]
[1999, 1495]
[2007, 1489]
[1965, 1459]
[2011, 1495]
[2002, 1499]
[2047, 1509]
[2002, 1451]
[2006, 1498]
[2003, 1493]
[2003, 1462]
[1957, 1491]
[2002, 1491]
[2000, 1494]
[2000, 1503]
[2000, 1495]
[2143, 1498]
[1998, 1490]
[2002, 1494]
[2007, 1499]
[2006, 1495]
[1999, 1495]
[1996, 1502]
[2004, 1497]
[2000, 1505]
[2006, 1486]
[2003, 1495]
[2005, 1499]
[2000, 1498]
[2004, 1494]
[1985, 1494]
[2000, 1523]
[2004, 1509]
[2003, 1494]
[1815, 1495]
[2006, 1495]
[2001, 1497]
[2001, 1498]
[2007, 1500]
[1802, 1497]
[2001, 1499]
[2009, 1499]
[2005, 1521]
[1983, 1515]
[2193, 1492]
[2001, 1498]
[1999, 1497]
[2000, 1493]
[2002, 1498]
[2013, 1497]
[2011,

In [10]:
ports = list(serial.tools.list_ports.comports())
for p in ports:
    desc = (p.description + str(p.manufacturer)).lower()
    print(desc)

silicon labs cp210x usb to uart bridge (com3)silicon labs
standard serial over bluetooth link (com15)microsoft
standard serial over bluetooth link (com4)microsoft
standard serial over bluetooth link (com10)microsoft
standard serial over bluetooth link (com21)microsoft
standard serial over bluetooth link (com11)microsoft
standard serial over bluetooth link (com13)microsoft
standard serial over bluetooth link (com16)microsoft
standard serial over bluetooth link (com22)microsoft
standard serial over bluetooth link (com5)microsoft
standard serial over bluetooth link (com6)microsoft


In [12]:
import serial
import time

# Replace 'COM3' with your ESP32 port if different
ser = serial.Serial('COM3', 115200, timeout=1)
time.sleep(1)  # wait for ESP32 to reset

try:
    while True:
        if ser.in_waiting:
            line = ser.readline().decode('utf-8', errors='ignore').strip()
            if line:
                print(line)
        time.sleep(0.01)

except KeyboardInterrupt:
    print("Stopped by user")
finally:
    ser.close()


[0,1,1,1,0,0,0,0,1045,672,388,573,2003,1498]
[0,1,1,1,0,0,0,0,1279,1109,880,1015,2001,1494]
[0,1,1,1,0,0,0,0,880,503,222,423,1997,1495]
[0,1,1,1,0,0,0,0,1319,1141,919,1074,2011,1519]
[0,1,1,1,0,0,0,0,848,473,192,386,2005,1489]
[0,1,1,1,0,0,0,0,1358,1181,970,1120,1997,1490]
[0,1,1,1,0,0,0,0,792,421,141,321,2001,1501]
[0,1,1,1,0,0,0,0,1390,1215,999,1167,2035,1499]
[0,1,1,1,0,0,0,0,814,435,150,345,2000,1487]
[0,1,1,1,0,0,0,0,1435,1251,1065,1231,2002,1497]
[0,1,1,1,0,0,0,0,774,397,116,302,2013,1501]
[0,1,1,1,0,0,0,0,1476,1299,1116,1281,1983,1499]
[0,1,1,1,0,0,0,0,724,354,66,240,1995,1495]
[0,1,1,1,0,0,0,0,1520,1360,1167,1338,2011,1495]
[0,1,1,1,0,0,0,0,651,293,2,157,2000,1503]
[0,1,1,1,0,0,0,0,1584,1395,1207,1392,2006,1499]
[0,1,1,1,0,0,0,0,708,333,48,216,2032,1495]
[0,1,1,1,0,0,0,0,1617,1429,1264,1446,2009,1495]
[0,1,1,1,0,0,0,0,684,304,19,185,2001,1495]
[0,1,1,1,0,0,0,0,1663,1483,1311,1496,2000,1500]
[0,1,1,1,0,0,0,0,574,213,0,63,2000,1498]
[0,1,1,1,0,0,0,0,1712,1533,1367,1568,2000,1495]